In [3]:
import os


In [6]:
%pwd

'c:\\Users\\adminn\\Documents\\GitHub\\Computer_vision_deployement'

In [5]:
os.chdir("../")

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:

    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path
    

In [8]:
from CnnClassifier.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH
from CnnClassifier.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:

    def __init__(self,
                 config_file_path=CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH):
        
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:

        config=self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(root_dir=config.root_dir,
        source_url=config.source_url,
        local_data_file=config.local_data_file,
        unzip_dir=config.unzip_dir)

        return data_ingestion_config

In [16]:
import urllib.request as request
import zipfile
import gdown
from CnnClassifier import logger
from CnnClassifier.utils.common import get_size

In [17]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):

        self.config=config

    def download_file(self)->str:

        '''Fetch data from the url'''

        try:
            dataset_url=self.config.source_url
            zip_download_dir=self.config.local_data_file
            os.makedirs("artifacts/data_ingestion",exist_ok=True)
            logger.info(f'Downloading data from {dataset_url} into file {zip_download_dir}')

            file_id=dataset_url.split('/')[-2]
            prefix="https://drive.google.com/uc?/export=download&id="

            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"downloaded data from from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e

    def extract_zip_file(self):

        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)



In [18]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e


[2023-12-30 21:19:41,000: INFO: common: yaml file: config\config.yaml load successfully]
[2023-12-30 21:19:41,003: INFO: common: yaml file: params.yaml load successfully]
[2023-12-30 21:19:41,005: INFO: common: created directory at: artifacts]
[2023-12-30 21:19:41,007: INFO: common: created directory at: artifacts/data_ingestion]
[2023-12-30 21:19:41,008: INFO: 3285660325: Downloading data from https://drive.google.com/file/d/1HSdfR_X_rcifwgICKkhe3zpcKCHgzZOa/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (uriginal): https://drive.google.com/uc?/export=download&id=1HSdfR_X_rcifwgICKkhe3zpcKCHgzZOa
From (redirected): https://drive.google.com/uc?/export=download&id=1HSdfR_X_rcifwgICKkhe3zpcKCHgzZOa&confirm=t&uuid=29b90af2-1c13-4bf6-80ac-bdbb744c219a
To: c:\Users\adminn\Documents\GitHub\Computer_vision_deployement\artifacts\data_ingestion\data.zip
100%|██████████| 49.0M/49.0M [00:08<00:00, 5.53MB/s]

[2023-12-30 21:19:51,076: INFO: 3285660325: downloaded data from from https://drive.google.com/file/d/1HSdfR_X_rcifwgICKkhe3zpcKCHgzZOa/view?usp=sharing into file artifacts/data_ingestion/data.zip]
